In [1]:
import numpy as np

from pw_scf import plane_wave_basis
from pw_scf import get_gth_pseudopotential

from pyscf.pbc import gto

In [2]:
def print_gth_parameters(cell, basis):

    """

    print the GTH pseudopotential parameters

    :param cell: the unit cell
    :param basis: planewave basis information (contains gth parameters)

    """

    print('')
    print('    ==> GTH pseudopotential parameters (local) <==')
    print('')

    params = basis.gth_params

    natom = len(cell._atom)

    for center in range (0, natom):
        print('        atom: %20s' % ( cell._atom[center][0] ) )
        print('        Zion: %20i' % ( params.Zion[center] ) )
        print('        rloc: %20.12lf' % ( params.rloc[center] ) )
        print('        c1:   %20.12lf' % ( params.local_cn[center][0] ) )
        print('        c2:   %20.12lf' % ( params.local_cn[center][1] ) )
        print('        c3:   %20.12lf' % ( params.local_cn[center][2] ) )
        print('        c4:   %20.12lf' % ( params.local_cn[center][3] ) )
        print('')

    print('    ==> GTH pseudopotential parameters (non-local) <==')
    print('')

    for center in range (0, natom):

        print('        atom: %20s' % ( cell._atom[center][0] ) )
        rl = params.rl[center]
        print('        rl:   %20.12lf %20.12lf %20.12lf' % ( rl[0], rl[1], rl[2] ) )
        print('        h^l_ij:')

        for l in range (0, 3):
            h = params.hgth[center]
            print('')
            print('            l = %i' % ( l ) )
            for i in range (0, 3):
                print('            %20.12lf %20.12lf %20.12lf' % ( h[l, i, 0], h[l, i, 1], h[l, i, 2] ) )
        print('')

In [3]:
# build a unit cell
a = np.eye(3) * 4.0
atom = 'Si 0 0 0'

cell = gto.M(a = a,
             atom = atom,
             unit = 'bohr',
             basis = 'sto-3g',
             pseudo = 'gth-blyp',
             precision = 1.0e-8,
             dimension = 3)
cell.build()

In [4]:
# get plane wave basis information
basis = plane_wave_basis(cell,                            
                         ke_cutoff = 500.0 / 27.21138602,
                         n_kpts = [1, 1, 1],
                         use_pseudopotential = True)

print("")
print('    no. k-points:                                %20i' % ( len(basis.kpts) ) )
print('    KE cutoff (eV)                               %20.2f' % ( basis.ke_cutoff * 27.21138602 ) )
print('    no. basis functions (orbitals, gamma point): %20i' % ( basis.n_plane_waves_per_k[0] ) )

In [5]:
# print gth pseudopotential parameters
#print_gth_parameters(cell, basis)

In [6]:
# grab the gth pseudopotential parameters (local):
# Zion, rloc, c1, c2, c3, c4

natom = len(cell._atom)

for center in range (0, natom):
    Zion = basis.gth_params.Zion[center]
    rloc = basis.gth_params.rloc[center]
    c1 = basis.gth_params.local_cn[center][0]
    c2 = basis.gth_params.local_cn[center][1]
    c3 = basis.gth_params.local_cn[center][2]
    c4 = basis.gth_params.local_cn[center][3]

In [7]:
# grab the gth pseudopotential parameters (nonlocal):
# rl, hlij
for center in range (0, natom):
    rl = basis.gth_params.rl[center]
    h_lij = basis.gth_params.hgth[center]
     

In [8]:
# get local components of the pseudopotential matrix
for kid in range (0, len(basis.kpts) ):
    vl = get_gth_pseudopotential(basis, kid, pp_component = 'local')

In [9]:
# get local components of the pseudopotential matrix
for kid in range (0, len(basis.kpts) ):
    vnl = get_gth_pseudopotential(basis, kid, pp_component = 'nonlocal')